In [1]:
import glob, shutil, os, json, re, random
import numpy as np
# from subprocess import call
 

In [7]:
#Change the data_type
data_type = 'train'
CAP = 10

data_src_path = '/home/edgar/data/frame'
ann_path = '/home/edgar/data/frame/annotations'
OUT_PATH = "/home/edgar/yolov5/data/frames"
cpy_dest = f'/home/edgar/yolov5/data/frames/images/{data_type}'

with open(f"{data_type}{CAP}.txt", 'r') as fin:   
    file_list = [row.strip() for row in fin]      
    for afile in file_list:        
        _, file_name, img_number, clss = afile.split('/')
        cpy_dst = f"{cpy_dest}/{file_name}-{img_number}"        
        if not os.path.exists(cpy_dst):            
            cpy_src = os.path.join(data_src_path, afile[:-2])            
            shutil.copy(cpy_src, cpy_dst) 
        img_name = file_name + "-" + img_number[:-4] 
        img_index = int(img_number[6:9]) - 1
        json_path = ann_path + "/" + afile.split('/')[1] + ".mp4.json"        
        with open(json_path) as json_file:
            data = json.load(json_file)
            imh = float(data['metadata']['height'])
            imw = float(data['metadata']['width'])
            bbox = data['annotations'][img_index]['bounding_box']            
            x = bbox['x']
            y = bbox['y']
            w = bbox['width']
            h = bbox['height']
            x_center = x + w / 2
            y_center = y + h / 2
            x_center /= imw
            y_center /= imh
            w = min(w / imw, .99)
            h = min(h / imh, .99)
            # w /= imw
            # h /= imh
            yolo_data = [[int(clss), x_center, y_center, w, h]]            
            
            yolo_data = np.array(yolo_data)
            # print(yolo_data)
            # with open(os.path.join(OUT_PATH, f"labels/{data_type}/{img_name}.txt"), 'w') as f:
            #     f.write("{}".format(yolo_data))
            np.savetxt(
                os.path.join(OUT_PATH, f"labels/{data_type}/{img_name}.txt"),
                yolo_data,
                fmt=["%d", "%f", "%f", "%f", "%f"]
            )